<a href="https://colab.research.google.com/github/prernasinghal88/Are-you-sure-to-Insure-/blob/version-3/Insurer_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurer Project

## Import Libraries

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Download Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import data
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/greyatom/test.csv')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/greyatom/train.csv')
sample = pd.read_csv('/content/drive/My Drive/Colab Notebooks/greyatom/sample_submission.csv')

df is the dataset we will be running our analysis and later modelling
test is the dataset we will be using our model on

Some Basic info about our dataset

In [4]:
df.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,3433,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,0,7,MALAYSIA,0.0,17.82,NaN,31
1,4339,EPX,Travel Agency,Online,Cancellation Plan,0,85,SINGAPORE,69.0,0.00,NaN,36
2,34590,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,0,11,MALAYSIA,19.8,11.88,NaN,75
3,55816,EPX,Travel Agency,Online,2 way Comprehensive Plan,0,16,INDONESIA,20.0,0.00,NaN,32
4,13816,EPX,Travel Agency,Online,Cancellation Plan,0,10,"KOREA, REPUBLIC OF",15.0,0.00,NaN,29


Claim id the target variable which we will be predicting and rest others are features we will be using to predict claim.

## Exploratory Data Analysis (EDA)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50553 entries, 0 to 50552
Data columns (total 12 columns):
ID                      50553 non-null int64
Agency                  50553 non-null object
Agency Type             50553 non-null object
Distribution Channel    50553 non-null object
Product Name            50553 non-null object
Claim                   50553 non-null int64
Duration                50553 non-null int64
Destination             50553 non-null object
Net Sales               50553 non-null float64
Commision (in value)    50553 non-null float64
Gender                  14600 non-null object
Age                     50553 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 4.6+ MB


In [0]:
So there are total 11 features and one target variable. Out of these we have to select the best features out of these for building a correct model

### Profile Report

In [6]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

Number of variables,12
Number of observations,50553
Total Missing (%),5.9%
Total size in memory,4.6 MiB
Average record size in memory,96.0 B
Numeric,5
Categorical,6
Boolean,1
Date,0
Text (Unique),0
Rejected,0


### Basic Countplot 

In [0]:
sns.countplot('Claim',data=df)
plt.title('Claims')
plt.show()

As per this we can estimate that not even 1% claims were passed. This is a highly imbalanced data.

### Pair Plot

In [7]:
sns.pairplot(df, hue='Claim')

/usr/local/lib/python3.6/dist-packages/statsmodels/nonparametric/kde.py:487: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
/usr/local/lib/python3.6/dist-packages/statsmodels/nonparametric/kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2


### Agency Type vs Claim

In [0]:
f,ax=plt.subplots(1,2,figsize=(10,6))
df[['Agency Type','Claim']].groupby(['Agency Type']).mean().plot.bar(ax=ax[0])
ax[0].set_title('% claims for agency types')
sns.countplot('Agency Type',hue='Claim',data=df,ax=ax[1])
ax[1].set_title('Claims vs Agency Type')
plt.setp(ax[1].get_xticklabels(), rotation=90, horizontalalignment='right')
plt.show()

Claim rate from Agency Type 'Airlines" is 3.5% whereas Agency type "Travel Agent" is only 1%.'

### Claims and Age Analysis

In [11]:
f,ax=plt.subplots(1, figsize=(10,6))
sns.barplot(x='Age',y='Claim',data=df)
ax.set_title('Claims Rate by Age')

Text(0.5, 1.0, 'Claims Rate by Age')

### Claims and Product Name Analysis

In [0]:
claim_y = df[df['Claim'] ==1]
f,ax=plt.subplots(1, figsize=(10,6))
ax = sns.countplot(x='Product Name',data=claim_y)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right", fontsize=10)
plt.tight_layout()
plt.show()

In [0]:
df[['Product Name','Claim']].groupby(['Product Name']).mean().plot.bar()
plt.title('% claims for different products')
plt.show()